# Introdução ao Reconhecimento de Padrões, 2020.2, UFC/DETI
## Trabalho 3

Aluno : Thyago Freitas da Silva <br>
Matrícula : 392035

In [1]:
import numpy as np
import operator
import collections
import pandas
from random import randrange
from sklearn.metrics import confusion_matrix as cm

### Scatter matrix helpers

In [31]:
def calc_mean(data):
    return np.mean(np.array(data),axis=0)

def within_cluster(results,data,centroids,k):
    data = data.values.tolist()
    s_w = []
    n = len(data[0])
    for k_value in range(k):
        values = []
        for index,v in enumerate(results):
            if v == k_value:
                mul = np.transpose(np.array(data[index]))-np.array(centroids[k_value])
                values.append(np.outer(mul,np.transpose(mul)))
        sum_k = np.zeros((n,n))
        for v in values:
            sum_k += v
        s_w.append(sum_k)
    result = np.zeros((n,n))
    for s in s_w:
        result += s
    return result.tolist()

def between_cluster(data,centroids,k):
    data = data.values.tolist()
    n = len(data[0])
    u = calc_mean(data)
    s_b = []
    for k_value in range(k):
        n_k = data.count(k_value)
        first_term = np.transpose(np.array(centroids[k_value]) - u)
        s_b.append(np.outer(first_term,np.transpose(first_term)))
    result = np.zeros((n,n))
    for s in s_b:
        result += s
    return result.tolist()

### Implementacão do classificador "K-Means"

In [75]:
norm_p = lambda x,y,p : abs((x-y))**p
euclidian = lambda list1,list2: np.sqrt(sum(map(norm_p,list1,list2,[2]*len(list1))))

class KMeansClassifier:
    def __init__(self, K, R):
        self.K = K
        self.R = R
        self.centroids = []
    def fit_predict(self, x_train):
        n = x_train.shape[0]
        result = [0]*n
        old = [0]*n
        aux = 0
        stop = False
        #idx = np.random.choice(len(x_train), self.K, replace=False)
        #for i in idx:
        #    self.centroids.append(x_train.iloc[i,:].to_list())
        for i in range(self.K):
            self.centroids.append(x_train.iloc[i,:].to_list())
        while aux < self.R:
            for row in range(n):
                d = []
                for c in range(self.K):
                    d.append(euclidian(x_train.iloc[row,:].to_list(),self.centroids[c]))
                result[row] = (d.index(min(d)))
            converge = True
            for x,y in zip(old,result):
                if x!=y:
                    converge = False
            if converge:
                break
            for c_n, c in enumerate(self.centroids):
                values = []
                values.append(c)
                for index, r in enumerate(result):
                    if r == c_n:
                        values.append(x_train.iloc[index,:].to_list())
                self.centroids[c_n] = calc_mean(values)
            aux += 1
            for r in range(len(result)):
                old[r] = result[r]
        print("K : {0}, \t R : {1} \t Parou com R igual a : {2}".format(self.K,self.R,aux))
        return result

### Leitura da base "demartology"

In [18]:
data = pandas.read_csv("./data/dermatology.csv", header=None)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 35 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       366 non-null    int64 
 1   1       366 non-null    int64 
 2   2       366 non-null    int64 
 3   3       366 non-null    int64 
 4   4       366 non-null    int64 
 5   5       366 non-null    int64 
 6   6       366 non-null    int64 
 7   7       366 non-null    int64 
 8   8       366 non-null    int64 
 9   9       366 non-null    int64 
 10  10      366 non-null    int64 
 11  11      366 non-null    int64 
 12  12      366 non-null    int64 
 13  13      366 non-null    int64 
 14  14      366 non-null    int64 
 15  15      366 non-null    int64 
 16  16      366 non-null    int64 
 17  17      366 non-null    int64 
 18  18      366 non-null    int64 
 19  19      366 non-null    int64 
 20  20      366 non-null    int64 
 21  21      366 non-null    int64 
 22  22      366 non-null    in

#### Pre-processamento

Olhando o conteúdo do arquivo, foi notado que algumas linhas da coluna 33 (band-like infiltrate) apresentam valores marcados com "?" que atrapalham o resultado do algoritmo. Para remediar esse problema , decidi trocar os valores faltantes demarcados por "?" pela mediana da coluna 33, pois tal medida é menos sensível a outliers se comparado com a média, por exemplo.

In [19]:
def calc_median(array):
    values = []
    for v in array:
        if v != '?':
            v = int(v)
            values.append(v)
    return np.median(values)

median = calc_median(data.iloc[:,33])
data.iloc[:,33] = list(map(lambda value: median if value == '?' else value, data.iloc[:,33]))
data.iloc[:,33] = data.iloc[:,33].astype(np.int64)

#### Normalizar dados

In [20]:
def normalize(data):
    columns = data.shape[1] - 1 #subtrai 1 para descartar coluna com classes
    for c in range(columns):
        min_v = np.min(data.iloc[:,c])
        max_v = np.max(data.iloc[:,c])
        data.iloc[:,c] = data.iloc[:,c].apply(lambda v : (v - min_v)/(max_v - min_v))
    return data

data = normalize(data)

#### Separar dados em atributos e classes de saída

In [21]:
atributes = data.iloc[:,:34]
target = data.iloc[:,34]

In [ ]:
def J(data,R,K):
    result = []
    for k in range(1,K+1):
        values = 0
        for r in range(1,R+1):
            kmeans = KMeansClassifier(k,r)
            v = kmeans.fit_predict(data)
            sw = within_cluster(v,data,kmeans.centroids,k)
            sb = between_cluster(data,kmeans.centroids,k)
            s = np.outer(np.linalg.inv(sw),sb)
            values += s.trace()
        result.append(values/R)
    return result

In [73]:
print(J(atributes,30,5))

K : 1, 	 R : 1 	 Convergiu antes de R :
K : 1, 	 R : 2 	 Convergiu antes de R :
K : 1, 	 R : 3 	 Convergiu antes de R :
K : 1, 	 R : 4 	 Convergiu antes de R :
K : 1, 	 R : 5 	 Convergiu antes de R :
K : 1, 	 R : 6 	 Convergiu antes de R :
K : 1, 	 R : 7 	 Convergiu antes de R :
K : 1, 	 R : 8 	 Convergiu antes de R :
K : 1, 	 R : 9 	 Convergiu antes de R :
K : 1, 	 R : 10 	 Convergiu antes de R :
K : 1, 	 R : 11 	 Convergiu antes de R :
K : 1, 	 R : 12 	 Convergiu antes de R :
K : 1, 	 R : 13 	 Convergiu antes de R :
K : 1, 	 R : 14 	 Convergiu antes de R :
K : 1, 	 R : 15 	 Convergiu antes de R :
K : 1, 	 R : 16 	 Convergiu antes de R :
K : 1, 	 R : 17 	 Convergiu antes de R :
K : 1, 	 R : 18 	 Convergiu antes de R :
K : 1, 	 R : 19 	 Convergiu antes de R :
K : 1, 	 R : 20 	 Convergiu antes de R :
K : 1, 	 R : 21 	 Convergiu antes de R :
K : 1, 	 R : 22 	 Convergiu antes de R :
K : 1, 	 R : 23 	 Convergiu antes de R :
K : 1, 	 R : 24 	 Convergiu antes de R :
K : 1, 	 R : 25 	 Converg

KeyboardInterrupt: 

In [76]:
kmeans = KMeansClassifier(5,30)
v = kmeans.fit_predict(atributes)
print(v)

K : 5, 	 R : 30 	 Parou com R igual a : 10
[0, 1, 2, 3, 2, 0, 0, 2, 0, 0, 1, 0, 0, 1, 4, 0, 0, 3, 4, 0, 0, 0, 0, 4, 0, 3, 0, 0, 0, 4, 3, 0, 1, 1, 0, 0, 4, 0, 4, 1, 0, 0, 1, 0, 0, 4, 0, 0, 0, 4, 4, 0, 1, 1, 0, 1, 0, 4, 0, 0, 0, 1, 0, 1, 0, 4, 3, 0, 0, 1, 0, 0, 4, 0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 0, 0, 1, 1, 4, 3, 0, 0, 0, 0, 1, 4, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 4, 1, 1, 2, 0, 2, 4, 0, 0, 2, 0, 1, 0, 0, 0, 1, 1, 0, 0, 4, 0, 0, 1, 1, 2, 0, 0, 1, 0, 0, 2, 1, 0, 1, 1, 4, 2, 2, 0, 0, 0, 0, 0, 1, 1, 1, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 1, 3, 4, 0, 2, 2, 3, 3, 3, 2, 3, 3, 0, 2, 2, 3, 1, 3, 3, 0, 0, 0, 0, 0, 3, 4, 4, 4, 3, 3, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 3, 3, 3, 3, 3, 4, 2, 2, 4, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 3, 3, 3, 3, 1, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 4, 2, 2, 2, 3, 3, 3, 3, 0, 0, 0, 0, 0, 4, 4, 4, 0, 3, 1, 0, 0, 0, 3, 3, 3, 4, 4, 4, 4, 4, 3, 3,

In [78]:
print(len(kmeans.centroids))

5
